In [81]:
# !git clone https://github.com/One-Night-Miracle/Data-Science-Project-2-2021-2-Nowcasting.git

In [82]:
# !rm -rf /content/Data-Science-Project-2-2021-2-Nowcasting

In [83]:
# !mv '/content/Data-Science-Project-2-2021-2-Nowcasting/data/bkk_radar_images_dBZ' '/content/'
# !mv '/content/Data-Science-Project-2-2021-2-Nowcasting/data/bkk_radar_images_mask' '/content/'

In [84]:
# import gdown

# urls = ['https://drive.google.com/drive/folders/1cs3sv5raII2sD1Sbz2cad0BDX-MRJ_nS?usp=sharing', 'https://drive.google.com/drive/folders/1vDnxOsk2PLhX3YUW_bysa0t-Ap1YkiAO?usp=sharing']
# gdown.download_folder(urls[0], quiet=True)
# gdown.download_folder(urls[1], quiet=True)

In [85]:
# !mv '/content/bkk_radar_images_dBZ' '/content/Data-Science-Project-2-2021-2-Nowcasting/data/'
# !mv '/content/bkk_radar_images_mask' '/content/Data-Science-Project-2-2021-2-Nowcasting/data/'

In [86]:
# !unzip -q '/content/Data-Science-Project-2-2021-2-Nowcasting/data/bkk_radar_images_dBZ/*' -d '/content/Data-Science-Project-2-2021-2-Nowcasting/data/bkk_radar_images_dBZ/'
# !unzip -q '/content/Data-Science-Project-2-2021-2-Nowcasting/data/bkk_radar_images_mask/*' -d '/content/Data-Science-Project-2-2021-2-Nowcasting/data/bkk_radar_images_mask/'

In [87]:
# !rm /content/Data-Science-Project-2-2021-2-Nowcasting/data/bkk_radar_images_dBZ/*.zip
# !rm /content/Data-Science-Project-2-2021-2-Nowcasting/data/bkk_radar_images_mask/*.zip

In [88]:
# !pip install tensorboardX

In [89]:
import torch
from torch.optim import lr_scheduler
import pandas as pd
import numpy as np

import os, sys
sys.path.insert(0, '/content/Data-Science-Project-2-2021-2-Nowcasting/')

from utils.config import cfg
from utils.blocks.forecaster import Forecaster
from utils.blocks.encoder import Encoder
from utils.tools.ordered_easydict import OrderedDict
from utils.blocks.module import EF
from utils.loss import Weighted_mse_mae
from utils.train_and_test import train_and_test
from experiment.net_params import conv2d_params
from utils.blocks.module import Predictor

if not os.path.exists(cfg.GLOBAL.MODEL_SAVE_DIR):
    os.makedirs(cfg.GLOBAL.MODEL_SAVE_DIR)

## Train-Valid-Test Split

In [90]:
from utils.tools.train_test_split import *

train_test_split(cfg.ONM_PD.FOLDER_ALL, ratio=(0.8,0.05,0.15))

## Experiment Conv2d

In [91]:
batch_size = cfg.GLOBAL.BATCH_SIZE
max_iterations = 20000
test_iteration_interval = 2000
test_and_save_checkpoint_iterations = 2000

LR = 1e-4

criterion = Weighted_mse_mae().to(cfg.GLOBAL.DEVICE)

model = Predictor(conv2d_params).to(cfg.GLOBAL.DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-6)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=4000, gamma=0.7)
folder_name = "conv2d"

### Verifying model network

In [92]:
model

Predictor(
  (model): Sequential(
    (conv1_relu_1): Conv2d(5, 64, kernel_size=(7, 7), stride=(5, 5), padding=(1, 1))
    (relu_conv1_relu_1): ReLU(inplace=True)
    (conv2_relu_1): Conv2d(64, 192, kernel_size=(5, 5), stride=(3, 3), padding=(1, 1))
    (relu_conv2_relu_1): ReLU(inplace=True)
    (conv3_relu_1): Conv2d(192, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu_conv3_relu_1): ReLU(inplace=True)
    (deconv1_relu_1): ConvTranspose2d(192, 192, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (relu_deconv1_relu_1): ReLU(inplace=True)
    (deconv2_relu_1): ConvTranspose2d(192, 64, kernel_size=(5, 5), stride=(3, 3), padding=(1, 1))
    (relu_deconv2_relu_1): ReLU(inplace=True)
    (deconv3_relu_1): ConvTranspose2d(64, 64, kernel_size=(7, 7), stride=(5, 5), padding=(1, 1))
    (relu_deconv3_relu_1): ReLU(inplace=True)
    (conv3_relu_2): Conv2d(64, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_conv3_relu_2): ReLU(inplace=True)
    (conv3_3

### Verifying input/output shape

In [93]:
data = torch.randn(5, 4, 1, 480, 480)
output = model(data.cuda())
print(output.size())

torch.Size([20, 4, 1, 480, 480])


### Train and Test

In [ ]:
train_and_test(model, optimizer, criterion, exp_lr_scheduler, batch_size, max_iterations, test_iteration_interval, test_and_save_checkpoint_iterations, folder_name)

  0%|          | 0/20000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
 21%|██▏       | 4293/20000 [1:11:23<2:17:53,  1.90it/s]

## Evaluation

In [ ]:
# from utils.tools.evaluation import *
# from utils.tools.dataloader import BKKIterator
# import copy
# import time
# import pickle

# IN_LEN = cfg.BENCHMARK.IN_LEN
# OUT_LEN = cfg.BENCHMARK.OUT_LEN
# name = 'conv2d'

In [ ]:
# with torch.no_grad():
#     is_deeplearning_model = (torch.nn.Module in model.__class__.__bases__)
#     if is_deeplearning_model:
#         model.eval()
#     evaluator = Evaluation(seq_len=OUT_LEN, use_central=False)
#     bkk_iter = BKKIterator(pd_path=cfg.ONM_PD.RAINY_TEST,
#                                     sample_mode="sequent",
#                                     seq_len=IN_LEN + OUT_LEN,
#                                     stride=cfg.BENCHMARK.STRIDE)
#     model_run_avarage_time = dict()
#     model_run_avarage_time[name] = 0.0
#     valid_time = 0
#     while not bkk_iter.use_up:
#         valid_batch, valid_mask, sample_datetimes, _ = bkk_iter.sample(batch_size=1)
#         if valid_batch.shape[1] == 0:
#             break
#         if not cfg.EVALUATION.VALID_DATA_USE_UP and valid_time > cfg.EVALUATION.VALID_TIME:
#             break

#         valid_batch = valid_batch.astype(np.float32) / 255.0
#         valid_data = valid_batch[:IN_LEN, ...]
#         valid_label = valid_batch[IN_LEN:IN_LEN + OUT_LEN, ...]
#         mask = valid_mask[IN_LEN:IN_LEN + OUT_LEN, ...].astype(int)

#         if is_deeplearning_model:
#             valid_data = torch.from_numpy(valid_data).to(cfg.GLOBAL.DEVICE)

#         start = time.time()
#         output = model(valid_data)
#         model_run_avarage_time[name] += time.time() - start

#         if is_deeplearning_model:
#             output = output.cpu().numpy()

#         output = np.clip(output, 0.0, 1.0)

#         evaluator.update(valid_label, output, mask)

#         valid_time += 1
#     model_run_avarage_time[name] /= valid_time
#     evaluator.save_pkl(os.path.join(cfg.BENCHMARK.STAT_PATH, name + '.pkl'))

# with open(os.path.join(cfg.BENCHMARK.STAT_PATH, 'model_run_average_time.pkl'), 'wb') as f:
#     pickle.dump(model_run_avarage_time, f)

# ## took around 7 mins

In [ ]:
# !zip -r /content/Data-Science-Project-2-2021-2-Nowcasting/models_save.zip /content/Data-Science-Project-2-2021-2-Nowcasting/models_save

In [ ]:
# for p in os.listdir(os.path.abspath(cfg.BENCHMARK.STAT_PATH)):
#   if p != 'conv2d.pkl': continue
#   e = pickle.load(open(os.path.join(cfg.BENCHMARK.STAT_PATH, p), 'rb'))
#   _, _, csi, hss, _, mse, mae, balanced_mse, balanced_mae, _ = e.calculate_stat()
#   print(p.split('.')[0])
#   for i, thresh in enumerate(cfg.EVALUATION.THRESHOLDS):
#       print('thresh %.1f csi: average %.4f, last frame %.4f; hss: average %.4f, last frame %.4f;'
#             % (thresh, csi[:, i].mean(), csi[-1, i], hss[:, i].mean(), hss[-1, i]))

#   print(('mse: average %.2f, last frame %.2f\n' +
#       'mae: average %.2f, last frame %.2f\n'+
#       'bmse: average %.2f, last frame %.2f\n' +
#       'bmae: average %.2f, last frame %.2f\n') % (mse.mean(), mse[-1], mae.mean(), mae[-1],
#             balanced_mse.mean(), balanced_mse[-1], balanced_mae.mean(), balanced_mae[-1]))